In [0]:

import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

from yellowbrick.model_selection import LearningCurve, learning_curve

from yellowbrick.classifier import ClassificationReport


In [0]:
with open('df_features_train.pickle', 'rb') as f:
    df_train = pickle.load(f)

In [0]:
with open('df_features_test.pickle', 'rb') as f:
    df_test = pickle.load(f)

In [0]:
#get training and testing data
X_train = df_train.drop(labels=['targets'], axis=1)
y_train = df_train['targets']

X_test = df_test.drop(labels=['targets'], axis=1)
y_test = df_test['targets']

In [0]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(10,10))
    ax.grid(False)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    #fig.tight_layout()
    return ax

In [0]:
class_names=np.array(['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal'])

# Random Forests

In [0]:
#instantiate the random forest
clf_Rf =RandomForestClassifier(n_estimators=20, max_depth=50, warm_start=True)

In [0]:
clf_Rf.fit(X_train, y_train)

In [0]:
visualizer = LearningCurve(clf_Rf, scoring='f1_weighted')
visualizer.fit(X_train, y_train)
visualizer.show(outpath="learningCurveRF.png")

In [0]:
y_pred_RF = clf_Rf.predict(X_test)

In [0]:
plot_confusion_matrix(y_test, y_pred_RF, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for Random Forest')
plt.savefig('confusionMatrixRFNormalized.png')
plot_confusion_matrix(y_test, y_pred_RF, classes=class_names, normalize=False,
                      title='Non-normalized confusion matrix for Random Forest')
plt.savefig('confusionMatrixRFNonNormalized.png')

In [0]:
visualizer = ClassificationReport(clf_Rf, support=True)
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show(outpath="classificationReportRF.png") 

In [0]:
accuracy_RF = np.mean(y_pred_RF == y_test)
print("The accuracy of Random Forest is {0:.2%}".format(accuracy_RF))

In [0]:
# pickle the trained model
with open("model_RF.pickle", mode='wb+') as file:
    pickle.dump(clf_Rf, file)

## Grid Search

In [0]:
param_dist = {"n_estimators" : [20, 40, 60, 80],
              "max_depth": [10, 20, 30, 40],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [0]:
#instantiate a new random forest
clf_RF_CV=RandomForestClassifier()

#set number of iterations
n_iter_search = 20
#creat the random search class
random_search_RF = RandomizedSearchCV(clf_RF_CV, param_distributions=param_dist,
                                      n_iter=n_iter_search, cv=5)

In [0]:
random_search_RF.fit(X_train, y_train)

In [0]:
y_pred_RF_random = random_search_RF.predict(X_test)

In [0]:
plot_confusion_matrix(y_test, y_pred_RF_random, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for Random Forest After Randomized Search')
plt.savefig('confusionMatrixRandomSearchRFNormalized.png')
plot_confusion_matrix(y_test, y_pred_RF_random, classes=class_names, normalize=False,
                      title='Non-normalized confusion matrix for Random Forest After Randomized Search')
plt.savefig('confusionMatrixRandomSearchRFNonNormalized.png')

In [0]:
random_search_RF.best_estimator_

In [0]:
visualizer = ClassificationReport(random_search_RF, support=True)
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show(outpath="classificationReportRandomSearchRF.png") 

In [0]:
accuracy_RF_random = np.mean(y_pred_RF_random == y_test)
print("The accuracy of Random Forest is {0:.2%}".format(accuracy_RF_random))

In [0]:
# pickle the trained model
with open("model_random_search_RF.pickle", mode='wb+') as file:
    pickle.dump(random_search_RF, file)

In [0]:
visualizer = LearningCurve(random_search_RF, scoring='f1_weighted')
visualizer.fit(X_train, y_train)
visualizer.show(outpath="learningCurveRandomSearchRF.png")